<a href="https://colab.research.google.com/github/asegura4488/MetodosComputacionales2026/blob/main/Semana6/CuadraturaGauss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import factorial
import sympy as sym

In [2]:
f = lambda x: 6*x**2

In [3]:
W = np.array([1.,1.])
X = np.array([-1./np.sqrt(3),1./np.sqrt(3)])

In [4]:
I = np.sum(W*f(X))
I

np.float64(4.000000000000001)

In [7]:
def GetNewtonMethod(f,df,xn,itmax = 1000, precision=1e-14):

    error = 1.
    it = 0

    while error >= precision and it < itmax:

      dfx = df(xn)

      # Evitar división por cero
      if np.abs(dfx) < 1e-16:
            print("Derivada cercana a cero. Método falla.")
            return False

      xn1 = xn - f(xn)/df(xn)
      error = np.abs( xn1-xn )

      xn  = xn1
      it += 1


    if it == itmax:
        return False
    else:
        return xn

In [8]:
def GetAllRoots(f, df, x, tolerancia_decimales=8, tol_unicidad=1e-6):

    Roots = []

    for x0 in x:

        root = GetNewtonMethod(f, df, x0)

        if root is not False:

            # Redondeamos
            croot = np.round(root, tolerancia_decimales)

            # Verificamos que no esté ya (con tolerancia)
            if not any(np.isclose(croot, r, atol=tol_unicidad) for r in Roots):
                Roots.append(croot)

    Roots = np.array(Roots)
    Roots.sort()

    return Roots

In [9]:
def GetLegendre(n):

  x = sym.Symbol('x',Real=True)
  #y = sym.Symbol('y',Real=True)

  y = (x**2 - 1)**n

  p = sym.diff(y,x,n)/(2**n * factorial(n))

  return p

In [10]:
GetLegendre(5)

0.125*x*(8*x**4 + 40*x**2*(x**2 - 1) + 15*(x**2 - 1)**2)

In [13]:
Legendre = []
DLegendre = []

x = sym.Symbol('x',Real=True)
n=6

for i in range(n+1):

    poly = GetLegendre(i)

    Legendre.append(poly)
    DLegendre.append(sym.diff(poly,x))

In [14]:
Legendre[n]

1.0*x**6 + 7.5*x**4*(x**2 - 1) + 5.625*x**2*(x**2 - 1)**2 + 0.3125*(x**2 - 1)**3

In [15]:
sym.diff(sym.exp(x),x)

exp(x)

In [16]:
def GetRootsPolynomial(n,xi,poly,dpoly):

    x = sym.Symbol('x',Real=True)
    pn = sym.lambdify([x],poly[n],'numpy')
    dpn = sym.lambdify([x],dpoly[n],'numpy')

    Roots = GetAllRoots(pn,dpn,xi)

    return Roots

In [17]:
xi = np.linspace(-1.,1,100,dtype=np.longdouble)
Roots = GetRootsPolynomial(n,xi,Legendre,DLegendre)
Roots

array([-0.93246951, -0.66120939, -0.23861919,  0.23861919,  0.66120939,
        0.93246951], dtype=float128)

In [18]:
# Aca calculamos los pesos de la cuadrtura

def GetWeights(Roots,DLegendre):

    Dpoly = sym.lambdify([x],DLegendre[n],'numpy')

    Weights = 2/( (1-Roots**2)*Dpoly(Roots)**2 )

    return Weights

In [20]:
Weights = GetWeights(Roots,DLegendre)
Weights

array([0.1713245 , 0.36076157, 0.46791393, 0.46791393, 0.36076157,
       0.1713245 ], dtype=float128)

In [24]:
# Calculando usando el paquete
a = 0
b = 0.25*np.pi
f = lambda x: np.sin(x)

In [34]:
Roots, Weights = np.polynomial.legendre.leggauss(10)
# Cambio de variable para un intervalo general
t = 0.5*( (b-a)*Roots + b + a )
Integral = 0.5*(b-a)*np.sum(Weights*f(t))
Roots

array([-0.97390653, -0.86506337, -0.67940957, -0.43339539, -0.14887434,
        0.14887434,  0.43339539,  0.67940957,  0.86506337,  0.97390653])

In [35]:
Weights

array([0.06667134, 0.14945135, 0.21908636, 0.26926672, 0.29552422,
       0.29552422, 0.26926672, 0.21908636, 0.14945135, 0.06667134])

In [36]:
print('{:.70f}'.format(Integral))
print('{:.70f}'.format(1-np.cos(0.25*np.pi)))

0.2928932188134524272626890706305857747793197631835937500000000000000000
0.2928932188134524272626890706305857747793197631835937500000000000000000


In [37]:
# Cuadratura de Laguerre
R,W = np.polynomial.laguerre.laggauss(7)
R

array([ 0.19304368,  1.0266649 ,  2.56787674,  4.90035308,  8.18215344,
       12.73418029, 19.39572786])